[Website](https://finance.yahoo.com/markets/stocks/most-active/)

In [2]:
import requests
import pandas as pd 
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [3]:
stock = []

path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.set_page_load_timeout(120)

try: 
    url = 'https://finance.yahoo.com/markets/stocks/most-active/?start=0&count=25' 
    driver.get(url) 

    for _ in range(10): 
        time.sleep(5)

        pager = driver.find_elements(By.TAG_NAME, 'tbody')

        for page in pager:
            rows = page.find_elements(By.TAG_NAME, 'tr')

            for row in rows:
                try:
                    data = row.find_elements(By.TAG_NAME, 'td')

                    row_data = {
                            "Symbol": data[0].text.strip(),
                            "Name": data[1].text.strip(),
                            "Price": data[3].text.strip(),
                            "Change": data[4].text.strip(),
                            "Change %": data[5].text.strip(),
                            "Volume": data[6].text.strip(),
                            "Avg Vol (3M)": data[7].text.strip(),
                            "Market Cap": data[8].text.strip(),
                            "P/E Ratio (TTM)": data[9].text.strip(),
                            "52 Wk Change %": data[10].text.strip()
                        }
                    stock.append(row_data) 

                except Exception as e:
                        print(f"Error parsing table: {e}")
                
        next_button = WebDriverWait(driver,10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='next-page-button']"))
        )
        next_button.click()

        time.sleep(5)
except:
    pass

finally:
     driver.quit()
            

In [7]:
j_path = "Most Active Stocks.json"

with open(j_path, 'w') as file:
    json.dump(stock, file, indent=4)

print(f"Data successfully saved to {j_path}")

Data successfully saved to Most Active Stocks.json


In [4]:
df = pd.DataFrame(stock)
df.to_csv('Most Active Stocks.csv')

In [5]:
df.head()

,Symbol,Name,Price,Change,Change %,Volume,Avg Vol (3M),Market Cap,P/E Ratio (TTM),52 Wk Change %
0,NVDA,NVIDIA Corporation,134.77,-4.04,-2.91%,167.414M,244.491M,3.301T,53.27,191.27%
1,WBA,"Walgreens Boots Alliance, Inc.",10.38,+1.51,+17.17%,94.171M,21.565M,8.958B,-,-61.40%
2,LCID,"Lucid Group, Inc.",2.3580,-0.1950,-7.6471%,92.809M,57.648M,7.063B,-,-39.57%
3,TSLA,"Tesla, Inc.",401.74,+11.95,+3.07%,87.698M,87.692M,1.291T,109.92,64.46%
4,SOUN,"SoundHound AI, Inc.",14.06,-1.00,-6.64%,84.171M,37.822M,5.188B,-,664.47%
